In [11]:
#nbi:hide_in
#nbi:hide_out
!pip3 install matplotlib
!pip3 install keras
!pip3 install spectral
!pip3 install seaborn
!pip3 install scikit-image
!pip3 install tensorflow
!pip3 install sklearn

     |████████████████████████████████| 10.3 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 53.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 47.0 MB/s eta 0:00:01
     |████████████████████████████████| 636 kB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 44.5 MB/s eta 0:00:01
     |████████████████████████████████| 27.4 MB 35.2 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 285 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 29.2 MB 138 kB/s eta 0:00:01
     |████████████████████████████████| 163 kB 57.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 30.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 62.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 57.5 MB/s eta 0:00:01
     |████████████████████████████████| 394.3 MB 8.4 kB/s  eta 0:00:01    |███▌                          

     |████████████████████████████████| 10.6 MB 36.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 61.6 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 71.9 MB/s eta 0:00:01██                   | 6.0 MB 71.9 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 31.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 52.4 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.0 MB/s  eta 0:00:01
     |████████████████████████████████| 129 kB 53.2 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 136 kB 39.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 8.2 MB/s  eta 0:00:01
     |████████████████████████████████| 298 kB 37.6 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 48.0 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 46.1 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 6.9 MB/

In [2]:
#nbi:hide_in
%matplotlib inline
import spectral as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from spectrum import rank_normalizer,scale_max_min,scale_white_dark,plot_spectra,zoom
from spectrum_ml import cluster_pixels_kmeans,detect_aphid,blob_detector
from ipywidgets import interact, interactive, widgets
from IPython.display import display
import io

In [9]:
chemin = widgets.Text()

options3 = {
    'title': 'Chemin_spectral',
}

def chm(x):
    chemin.value = str(x)

ch = interactive(chm, x = '/home/plavainne/Documents/BIM/nbinteract-tutorial/VL1grF/capture', options = options3)

In [10]:
display(ch)

interactive(children=(Text(value='/home/plavainne/Documents/BIM/nbinteract-tutorial/VL1grF/capture', descripti…

In [11]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['SPECTRAL_DATA'] = chemin.value

In [12]:
def ouvrir_csv(wl):
    #nomfichier=askopenfilename()
    return pd.read_csv(wl)

In [13]:
wl = widgets.FileUpload(
    accept='',  # A/home/plavainne/Documents/BIM/06022020/06022020/VL1grF/ccepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description = 'Wave_Length'
    #command = ouvrir_csv()
    #value=pd.read_csv
)

display(wl)

FileUpload(value={}, description='Wave_Length')

In [14]:
# Create text widget for output
exp_name1 = widgets.Text()
exp_name2 = widgets.Text()
exp_name3 = widgets.Text()
current_dir = widgets.Text()

In [15]:
options1 = {
    'title': 'Exp_name',
}

options2 = {
    'title': 'dossier courant',
}

In [16]:
# Define function to bind value of the input to the output variable 
def name1(x):
    exp_name1.value = str(x)

def name2(x):
    exp_name2.value = str(x)
    
def name3(x):
    exp_name3.value = str(x)
    
def dossier_courant(x):
    current_dir.value = str(x)

In [17]:
en1 = interactive(name1, x = "VL1grF.hdr", options = options1)
en2 = interactive(name2, x = "DARKREF_VL1grF.hdr", options = options1)
en3 = interactive(name3, x = "WHITEREF_VL1grF.hdr", options = options1)

In [20]:
cd = interactive(dossier_courant, x = "/home/plavainne/Documents/BIM/nbinteract-tutorial/")

In [21]:
display(en1, options = options1)

interactive(children=(Text(value='VL1grF', description='x'), Output()), _dom_classes=('widget-interact',))

In [ ]:
display(en2, options = options1)

In [ ]:
display(en3, options = options1)

In [22]:
display(cd)

interactive(children=(Text(value='/home/plavainne/Documents/BIM/06022020/06022020/', description='x'), Output(…

In [23]:
button = widgets.Button(description="Rechargez les donnees")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        
        wave_lengths = pd.read_csv(io.BytesIO(wl.value[list(wl.value.keys())[0]]['content']), header=None, index_col=0)
        image_path = exp_name1.value#current_dir.value+exp_name1.value+'/capture/'+exp_name1.value+'.hdr'
        dark_ref_path = exp_name2.value#current_dir.value+exp_name.value+'/capture/DARKREF_'+exp_name.value+'.hdr'
        white_ref_path = exp_name3.value#current_dir.value+exp_name.value+'/capture/WHITEREF_'+exp_name.value+'.hdr'
        
        img = sp.open_image(image_path)

        dark_ref = sp.open_image(dark_ref_path)
        dark_spectrum = dark_ref.asarray().reshape(-1,dark_ref.shape[2]).mean(axis=0)

        white_ref = sp.open_image(white_ref_path)
        white_spectrum = white_ref.asarray().reshape(-1,white_ref.shape[2]).mean(axis=0)
        
        img_norm = scale_white_dark(img.asarray(),dark_spectrum=dark_spectrum,white_spectrum=white_spectrum)
        img_norm = scale_max_min(img_norm)
        
        img_norm_reshape = img_norm.reshape(-1,img_norm.shape[-1])
        
        print(img.shape)
        plot_spectra(img,  wave_lengths=(29,113,227), grid_step=20)

button.on_click(on_button_clicked)

Button(description='Rechargez les donnees', style=ButtonStyle())

Output()